In [49]:
# Retrieval-Augmented Generation (RAG) System 

import os
from langchain_community.document_loaders import PyPDFLoader 
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.chat_models import ChatOpenAI
from langchain.chains.conversation.memory import ConversationBufferWindowMemory
from langchain.chains import ConversationalRetrievalChain

In [50]:
#constants
# OpenAI API key
OPENAI_API_KEY = "sk.."  
# provide pdf file path
PDF_PATH = "/Users/ANDHAWAN/Downloads/pyProjects/LLM QA/Benefits2024.pdf" 
#local directory path for vector db to store embeddings
VECTOR_DB_DIRECTORY = "/Users/ANDHAWAN/Downloads/pyProjects/LLM QA"
#gpt model you want to use
GPT_MODEL_NAME = 'gpt-4'

#performance parameters
CHUNK_SIZE = 1000
CHUNK_OVERLAP = 50


In [51]:
def load_and_split_document(pdf_path):
    #loads and splits the document into pages.
    loader = PyPDFLoader(pdf_path)
    return loader.load_and_split()

def split_text_into_chunks(pages, chunk_size, chunk_overlap):
    #splits text into chunks
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    return text_splitter.split_documents(pages)

def create_embeddings(api_key):
    #initializes embeddings
    return OpenAIEmbeddings(openai_api_key=api_key)

def setup_vector_database(documents, embeddings, directory):
    #sets up a vector database to store embeddings
    if not os.path.exists(directory):
        os.makedirs(directory)
    return Chroma.from_documents(documents=documents, embedding=embeddings, persist_directory=directory)

def initialize_chat_model(api_key, model_name):
    #initializes openaichat model
    return ChatOpenAI(openai_api_key=api_key, model_name=model_name, temperature=0.0)

def create_retrieval_qa_chain(chat_model, vector_database):
    #creates a retrieval QA chain combining model and database."""
    memory = ConversationBufferWindowMemory(memory_key='chat_history', k=5, return_messages=True)
    return ConversationalRetrievalChain.from_llm(chat_model, retriever=vector_database.as_retriever(), memory=memory)

def ask_question_and_get_answer(qa_chain, question):
    #asks questions
    return qa_chain({"question": question})['answer']

In [52]:
#
pages = load_and_split_document(PDF_PATH)
documents = split_text_into_chunks(pages, CHUNK_SIZE, CHUNK_OVERLAP)
embeddings = create_embeddings(OPENAI_API_KEY)
vector_database = setup_vector_database(documents, embeddings, VECTOR_DB_DIRECTORY)
chat_model = initialize_chat_model(OPENAI_API_KEY, GPT_MODEL_NAME)
qa_chain = create_retrieval_qa_chain(chat_model, vector_database)

In [55]:

questions = [
    "when will Gypass be available",
    "How much is frame allowence",
    "How many vision plans are available",
    "do we have leagl insurance"
]

# Process Questions and Answers
for question in questions:
    answer = ask_question_and_get_answer(qa_chain, question)
    print(f"Question: {question}\nAnswer: {answer}\n")


Question: whed will Gypass be available
Answer: Gympass will be available beginning on January 1, 2024.

Question: How much is frame allowence
Answer: The frame allowance for Vision Plan I will increase from $150 to $200 and for Vision Plan II from $200 to $300.

Question: How many vision plans are available
Answer: There are two available vision plans.

Question: do we have leagl insurance
Answer: Yes, ARAG is providing legal insurance services. They are expanding their services in 2024 to include more support for elder law issues such as Medicare eligibility or Social Security, and preparation of legal documents like deeds, mortgages, affidavits, wills, and power of attorney for your aging parents and grandparents. They also offer help in assessing eldercare needs and developing care plans.

